# An example of neutrino propagation simulation through flexOPT.jl 
Some of functions are the fruit of works from Estelle Salomé (July-August 2025) and TIPE (2025-2026).

i need to make a lazy option (not only regarding DrWatson)

In [1]:

using Pkg

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv" # 1D Earth/Mars/Moon models are defined

include("../src/commonBatchs.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

include("../src/Neurthino.jl")

using .commonBatchs, .planet1D, .GeoPoints
using .Neurthino 
using Colors



  Activating project at `~/Documents/Github/flexOPT`


if you want to use not decimated value for U and H or you want to propose any other flavour configuration, do this!

`U, H=set_default_flavour!(:NormalThree;roundingU=0.e0,roundingH=0.e0) `

geodynamical model directory (Takashi \& Paul, we cannot share the models online!)
Paul is ready to give us more models whenever we want but the data are confidential and I cannot share on the Github

In [ ]:
dir="/Users/nobuaki/Documents/MantleConvectionTakashi/op_old_full_mars_2025/"
rhoFiles=myListDir(dir; pattern=r"test_rho\d");
compositionFiles=myListDir(dir; pattern=r"test_c\d");
temperatureFiles=myListDir(dir; pattern=r"test_t\d");
wtrFiles=myListDir(dir; pattern=r"test_wtr\d");

## making 2D(/3D) box with the aid of GeoPoint.jl, reading 1D Earth model (cf. ParamFile) using planet1D.jl

Please refer to getRegionalBox.ipynb if you have some questions 

In [ ]:
set_default_planet!(:SphericalEarth) # the default is ":Earth"
#SphericalEarth means a perfect sphere with a radius of 6371000.0 m 
# otherwise of course, I made an Earth with ellipticity 

#planet1D.my1DDSMmodel=getSet1Dmodel!(:ak135) # if you want to try other 1D models, you can call getSet1Dmodel!(:name; modelPath="model_0000.nd")

#planet1D.my1DDSMmodel=getSet1Dmodel!(:PREM) 

In [ ]:
# you can do some more exercises with getRegionalBox.ipynb but just trust me here

# two (extreme) points that can define the slice (or the x-y local plane for 3D box)
p1 = GeoPoint(0.0,0.0;alt=30.e3) # lat, lon in degrees, altitude in metre (as an option)
# if you call GeoPoints with three floats without ; for alt, it will interpret as x, y, z in cartesian ecef coordinates
p2 = GeoPoint(0.0,180.0;alt=30.e3) # 


Δx = 25.e3 # in metre
Δz = 25.e3

altMax = 6500.e3
altMin = -6500.e3

as you can see in the GeoPoints.jl example, we can construct 3D box too, which can be maybe a good idea to explore the uncertainties of azimuth of incident angle.

In [ ]:
boxGrids=constructLocalBox(p1,p2,Δx,Δz,altMin,altMax;centreOption ="centreOfPlanet")

In [ ]:
# getting model parameters on the 2D disk
seismicModel=lazyProduceOrLoad("seismicModel2DGlobal",getParamsWithoutTopo,boxGrids.allGridsInGeoPoints,boxGrids.effectiveRadii)

In [ ]:
using CairoMakie
xvals = [p.xz[1] for p in boxGrids.allGridsInCartesian[:,1]]*1.e-3
zvals = [p.xz[2] for p in boxGrids.allGridsInCartesian[1,:]]*1.e-3
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    #collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel.ρ;
    xvals, zvals, seismicModel.ρ ;
    colormap = :inferno,
    #colorrange=(0,4),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "altitude from p1", title = "Density model")
)
Colorbar(fig[1,2], hm, label="density")
fig

# Superposition or Replacement by tomographic/geodynamic 2D/3D models

Tell me if you have your own format of the model, I will try to make buffer functions

# Reading models from Takashi and Paul